In [1]:
import import_ipynb
import sys
import torch
from torch import nn
from torch import optim                                        

if torch.cuda.is_available():
    cuda = torch.cuda.device("cuda:0")

%cd /home/ankit/Commitments/earthquake-prediction/

from preprocessing import *
from training import *
from postprocessing import *
from testing import *

%cd /home/ankit/Commitments/earthquake-prediction/Indonesia-Sulawesi/

/home/ankit/Commitments/earthquake-prediction
importing Jupyter notebook from preprocessing.ipynb
importing Jupyter notebook from training.ipynb
torch.Size([10, 1, 9])
torch.Size([1, 9])
torch.Size([10, 1, 9])
torch.Size([1, 9])
torch.Size([10, 1, 9])
torch.Size([1, 90])
importing Jupyter notebook from postprocessing.ipynb
importing Jupyter notebook from testing.ipynb
/home/ankit/Commitments/earthquake-prediction/Indonesia-Sulawesi


In [21]:
# SETTINGS D

data_file = "data.csv"
time_parser = time_from_string_dmy
mag_range = (4, 5)
params_file = "light-cnn-lstm.params"
rows = 3
cols = 3
total_params = rows*cols
time_intervals = 1200
train_size = 0.8
lookback_window = 50

In [22]:
earthquakes = extract_from_csv(data_file, time_parser, mag_range)
cells = Spatio_Temporal_Segmentation(rows, cols, time_intervals, earthquakes)

print(cells[0][0][0])

No earthquakes were recorded in this cell.


In [23]:
generator = TensorGen(cells)

eventtensor = generator.eventNoEvent()
# eventtensor = eventtensor.long()
print(eventtensor.dtype)
freqtensor = generator.freq()
minmagtensor = generator.min_magnitude()
maxmagtensor = generator.max_magnitude()

if torch.cuda.is_available:
    eventtensor = eventtensor.cuda()
    freqtensor = freqtensor.cuda()
    minmagtensor = minmagtensor.cuda()
    maxmagtensor = maxmagtensor.cuda()
    
print(eventtensor.size())
print(freqtensor.size())
print(minmagtensor.size())
print(maxmagtensor.size())

torch.float32
torch.Size([1200, 1, 9])
torch.Size([1200, 1, 9])
torch.Size([1200, 1, 9])
torch.Size([1200, 1, 9])


<strong>TRAINING EVENT PREDICTOR</strong>

In [24]:
#SETTINGS M1

hidden_parameters = 40
linear_parameters = 20
num_layers = 1

In [25]:

model = CNN_LSTM(in_channels=lookback_window, out_channels=80, kernel_size=3, in_parameters=rows*cols, hidden_parameters=hidden_parameters, num_layers=num_layers, activation=nn.Tanh())
# model.load_state_dict(torch.load(params_file))
if torch.cuda.is_available():
    model = model.cuda()
    
# out = model(eventtensor[:10])
# print(out)

In [26]:
# SETTINGS T1
pos_weight=torch.tensor([1])
# weight = None

# print(weight)
pos_weight = pos_weight.cuda()
loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
loss_function = nn.MSELoss()
n_epochs = 100

In [42]:
optimizer = optim.Adagrad(model.parameters(), lr=0.001)

# tqdm._instances.clear()
train(model, eventtensor, optimizer, loss_function, n_epochs=1)

100%|██████████| 1150/1150 [00:05<00:00, 204.05it/s]


Epoch 1 completed. 
Training Loss: 0.144
Test Loss: 0.128


In [43]:
# params_file = "light-cnn-lstm.params"
# torch.save(model.state_dict(), params_file)

In [44]:
evaluate(data=eventtensor, model=model, train_size=0.8, lookback_window=lookback_window, threshold=0.5)

100%|██████████| 1150/1150 [00:02<00:00, 391.40it/s]


Evaluation results for training dataset:
8190 evaluations in total.
3493/3900 cells with earthquake activity were correctly identified. (89.56 %)
2931/4290 cells with no earthquake activity were correctly identified. (68.32 %)
1359/4852 positives turned out to be false.
407/3338 negatives turned out to be false.
Preicision: 0.720
Recall: 0.896
F1 Score: 0.798
Overall accuracy: 78.44 %
Evaluation results for training dataset:
2160 evaluations in total.
1509/1628 cells with earthquake activity were correctly identified. (92.69 %)
361/532 cells with no earthquake activity were correctly identified. (67.86 %)
171/1680 positives turned out to be false.
119/480 negatives turned out to be false.
Preicision: 0.898
Recall: 0.927
F1 Score: 0.912
Overall accuracy: 86.57 %
